# Создание шаблона

## Установка и импорт

In [ ]:
!pip install docxtpl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install python-docx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from docxtpl import DocxTemplate
import os
import pandas as pd
import docx
from datetime import datetime
from datetime import date

## Загрузка данных

In [ ]:
excel_data = pd.read_excel('data.xlsx')
excel_data

,Фамилия,Имя,Отчество,Группа,Кол-во долгов
0,Антонова,Анна,Алексеевна,6031,4
1,Козлова,Мария,Олеговна,6032,6
2,Александров,Иван,Иванович,6032,5
3,Петров,Иван,Вассарионович,6032,1


## Работа с данными
Загрузка шаблона

In [ ]:
doc = docx.Document("sablon_uved.docx")
text = []
for p in doc.paragraphs:
    text.append(p.text)
text

['',
 'Обучающемуся (-ейся)',
 '{{fio1}}',
 'группы {{ group }}',
 '',
 '',
 'Уведомление о задолженностях',
 '№ {{num_u}}',
 'Уважаемый (-ая) {{fio2}}',
 '',
 'Уведомляем Вас о том, что на {{cur_date}} у вас имеется {{n}}.',
 '',
 '',
 'Уведомление получил:',
 'Обучающийся\t  _____________________ /__________________________________',
 '(Подпись)\t\t (ФИО обучающегося полностью)',
 '',
 '',
 '']

Находим сегодняшнюю дату:

In [ ]:
current_date = date.today().strftime("%d.%m.%Y")
current_date

'02.03.2023'

Нам нужно создать столько документов, сколько строк в таблице, следовательно, количество интераций будет совпадать с размерностью таблицы: (1 число - количество строк, 2 число - количество столбцов, нам нужно первое)

In [ ]:
excel_data.shape

(4, 5)

In [ ]:
len_data = excel_data.shape[0]
len_data

4

In [ ]:
numb = 25

for i in range(len_data):  #цикл считывания информации, вывода, создания уведомлений
    doc = DocxTemplate("sablon_uved.docx")
    second_name = excel_data['Фамилия'][i]
    first_name = excel_data['Имя'][i]
    patronymic = excel_data['Отчество'][i]
    n = excel_data['Кол-во долгов'][i]
    group = excel_data['Группа'][i]

    print(second_name, first_name, patronymic, n, group) #Выводим на экран для проверки
    fio1 = f'{second_name} {first_name} {patronymic}' #ФИО

    num_u = str(numb) + '-22/23' # получаем номер уведомления

    # подставляем в переменные шаблона значения из программы:
    context = {'fio1': fio1, 'fio2': fio1,  'group': group, 'n': str(n) +' задолженностей',
               'num_u': num_u, 'cur_date': current_date }


    doc.render(context)
    # сохраняем документ, каждый документ будет иметь следующий формат названия NФамилия.docx
    doc.save(str(i+1) + str(excel_data['Фамилия'][i])+'.docx')

    numb += 1

Антонова Анна Алексеевна 4 6031
Козлова Мария Олеговна 6 6032
Александров Иван Иванович 5 6032
Петров Иван Вассарионович 1 6032


## Выводы
Создали код, который заполняет шаблон, однако у шаблона есть ряд недостатков. Он требует падежей от вставляемых данных. Следовательно, можно его изменить под неизменяемый падеж.

# Использование `pymorphy2`

## Установка и импорт

In [ ]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer() # морфологический анализ

## Работа с данными:

In [ ]:
numb = 25

len_data = excel_data.shape[0] # количество студентов

current_date = date.today().strftime("%d.%m.%Y") # текущая дата

for x in range(len_data): # цикл по числу студентов
    doc = DocxTemplate("sablon_uved.docx")

    # Получение имени
    name = excel_data['Имя'][x].strip() # strip используется для удаления пробелов, если вдруг в слове будет лишний пробел
    name1 = morph.parse(name)[0] # получение первого разбора для имени
    gender = name1.tag.gender # получение пола, к которому относится имя
    name1 = name1.inflect({'datv'}).word.title() # склонение имени в дательный падеж, title - для написания с большой буквы

    # Аналогично получение фамилии:
    second_name = excel_data['Фамилия'][x].strip()
    second_name1 = morph.parse(second_name)[0]
    second_name1 = second_name1.inflect({'sing','datv',  gender}).word.title()
    # sing - единственной число, datv - дательный падеж, gender - указание пола, к которому относилось имя
    # это необходимо, чтоб избежать неверного склонения такой фамилии как Сидорова.

    # Аналогично получение отчества:
    patronymic = excel_data['Отчество'][x].strip()
    patronymic1 = morph.parse(patronymic)[0]
    patronymic1 = patronymic1.inflect({'datv'}).word.title()

    n = excel_data['Кол-во долгов'][x] # получение количества долгов
    group = excel_data['Группа'][x] # получения номера группы


    name1 = f'{second_name1} {name1} {patronymic1}' # ФИО в дательно падаже, чтобы подставлять в fio1
    name2 = f'{second_name} {name} {patronymic}' # ФИО в именительном падаже, чтобы подставлять в fio2

    # получение слово задолженность в нужной форме в зависимости от количества n
    n_ph = morph.parse('задолженность')[0]
    n_ph = n_ph.make_agree_with_number(n).word
    n_ph = f'{n} {n_ph}' # фраза, которая будет выводиться в документе "4 задолженности" и тд

    print(name2)
    numberu = str(numb) + '-21/22'

    context = {'fio1': name1, 'fio2': name2,  'group': group, 'n': n_ph,  'num_u': numberu, 'cur_date': current_date }

    doc.render(context)

    doc.save(str(x+1)+ str(excel_data['Фамилия'][x])+'.docx')
    numb += 1

Антонова Анна Алексеевна
Козлова Мария Олеговна
Александров Иван Иванович
Петров Иван Вассарионович


# Индивидуальное задание

**Ввод переменных с клавиатуры**:

- ФИО студента
- номер группы
- номер уведомления
- номер договора
- дата договора
- сумма задолженности



In [ ]:
list = []
list.append(['словами'])
list

[['словами']]

In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer() # морфологический анализ

In [ ]:
doc = docx.Document("Шаблон_Дутов.docx")
text = []
for p in doc.paragraphs:
    text.append(p.text)
text

['',
 'Обучающемуся (-ейся)',
 '{{fio_dative}}',
 'группы {{group}}',
 '',
 '',
 'Уведомление № {{num_u}}',
 'от {{cur_date}}',
 '',
 'Уведомляем Вас о том, что в настоящее время у Вас имеется задолженность по оплате за обучение по договору об обучении {{contract}} от {{con_date}} в размере {{debt}}.',
 '',
 'Директор института  \t  ________________ /_____{{director}}___________________',
 '(Подпись)\t\t     (ФИО)',
 '',
 '',
 '',
 'Уведомление получил:',
 'Обучающийся\t  _____________________ /__{{fio_nominative}}',
 '(Подпись)\t\t (ФИО обучающегося)',
 '',
 '',
 '']

Проверка некоторых вводных даных

In [ ]:
name = 'Дмитрий'
morph.parse(name)[0].inflect({'datv'}).word.title() #name_dative

'Дмитрию'

In [ ]:
second_name = 'Журавлёв'
gender = morph.parse(name)[0].tag.gender
morph.parse(second_name)[0].inflect({'sing','datv', gender}).word.title()

'Журавлёву'

Примечание: pymorphy2 не работает со всеми фамилиями, так как не является достаточно совершенной моделью. Например:

In [ ]:
second_name = 'Дутов'
gender = morph.parse(name)[0].tag.gender
morph.parse(second_name)[0].inflect({'sing','datv', gender}).word.title()

'Дуту'

In [ ]:
second_name = 'Дутову'
gender = morph.parse(name)[0].tag.gender
morph.parse(second_name)[0].inflect({'sing','nomn', gender}).word.title()

'Дутов'

In [ ]:
second_name = 'Журавлёву'
gender = morph.parse(name)[0].tag.gender
morph.parse(second_name)[0].inflect({'sing','nomn', gender}).word.title()

'Журавлёв'

Но мы видим, что слова, которые изначально подаются в дательном падеже, куда лучше становятся в именительный падеж, это также решает проблему несклоняемых неизвестных морфологизатору фамилий. Поэтому лучше попросить пользователя ввести фамилию в дательном подеже. Так как по условию данные вводят, а не используют из таблицы, мы можем рассчитывать на пользовательский ввод.

In [ ]:
n_ph = morph.parse('рубли')[0]
n_ph = n_ph.make_agree_with_number(2081).word
n_ph = f'{2081} {n_ph}'
n_ph

'2081 рубль'

fio_dative - ФИО в дательном падеже студента

group - группа студента

num_u - номер уведомления

cur_date - сегодняшняя дата

contract - номер договора

con_date -  дата договора

debt - сумма долга

director - И.О. Фамилия декана/директора

fio_nominative - И.О. Фамилия студента в именительном падеже

In [ ]:
current_date = date.today().strftime("%d.%m.%Y") # текущая дата

for x in range(int(input('Введите, для скольки студентов заполнять уведомление'))): # цикл по числу студентов
    doc = DocxTemplate("Шаблон_Дутов.docx")

    numb = input('Номер уведомления: ')
    contract = input('Введите номер договора: ')
    debt = int(input('Введите размер долга: '))
    director = input('Введите ФИО директора института/декана факультета: ') #у нас есть два варианта ввода: Иванов И.И. и Иванов Иван Иванович
    if director.count('.')==0:
      sub = director.split()
      director = f'{sub[0]} {sub[1][0]}. {sub[2][0]}.' #если у нас ввод полных ФИО, мы выводим Фамилия И.О.

    # Получение имени
    name = input('Введите имя студента:')
    name1 = morph.parse(name)[0] # получение первого разбора для имени
    gender = name1.tag.gender # получение пола, к которому относится имя
    name1 = name1.inflect({'datv'}).word.title() # склонение имени в дательный падеж, title - для написания с большой буквы

    # Аналогично получение фамилии:
    second_name1 = input('Введите фамилию студента в дательном (кому?) падеже: ')
    second_name = morph.parse(second_name1)[0]
    second_name = second_name.inflect({'sing','nomn', gender}).word.title()
    # sing - единственной число, nomn - именительный падеж падеж, gender - указание пола, к которому относилось имя

    # Аналогично получение отчества:
    patronymic = input('Введите отчество студента: ')
    patronymic1 = morph.parse(patronymic)[0]
    patronymic1 = patronymic1.inflect({'sing','datv', gender}).word.title()

    n = excel_data['Кол-во долгов'][x] # получение количества долгов
    group = input('Введите номер группы: ') # получения номера группы


    name1 = f'{second_name1} {name1} {patronymic1}' # ФИО в дательно падаже, чтобы подставлять в fio_dative
    name2 = f'{second_name} {name[0]}. {patronymic[0]}.' # ФИО в именительном падаже, чтобы подставлять в fio2

    # получение слово задолженность в нужной форме в зависимости от количества n
    n_ph = morph.parse('рубли')[0]
    n_ph = n_ph.make_agree_with_number(debt).word
    n_ph = f'{debt} {n_ph}' # фраза, которая будет выводиться в документе "4 задолженности" и тд

    print(name2)
    numberu = str(numb) + '-22/23'

    context = {'fio_dative': name1,
               'fio_nominative': name2,
               'group': group,
               'num_u': numberu,
               'cur_date': current_date,
               'contract':contract,
               'debt':n_ph,
               'director':director}

    doc.render(context)

    doc.save('test'+'.docx')

Введите, для скольки студентов заполнять уведомление1
Номер уведомления: 0001
Введите номер договора: 121212
Введите размер долга: 5452
Введите ФИО директора института/декана факультета: Лосев Константин Викторович
Введите имя студента:Антон
Введите фамилию студента в дательном (кому?) падеже: Попову
Введите отчество студента: Викторович
Введите номер группы: 6359
Попов А. В.
